In [6]:
# Install packages using conda first (for those available in conda)
import sys
import subprocess

def install_packages():
    # Try conda first for available packages
    print("Installing packages with conda...")
    subprocess.check_call([sys.executable, "-m", "conda", "install", "-y", "flask", "requests"])
    
    # Use pip for packages not available in conda
    print("Installing remaining packages with pip...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "flask-cors", "python-chess"])
    
    print("Installation complete!")

install_packages()

Installing packages with conda...
Installing remaining packages with pip...
Installation complete!


In [ ]:
from flask import Flask, request, jsonify
import chess
import threading

app = Flask(__name__)

# Game state storage
games = {}
move_history = {}
lock = threading.Lock()

def get_ai_move(fen, selected_llm="gemini", difficulty="normal"):
    """Placeholder AI logic (Replace with actual LLM API call)"""
    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)

    if not legal_moves:
        return None  # No legal moves left (Checkmate/Stalemate)

    if difficulty == "easy":
        move = legal_moves[0]  # Picks the first legal move
    elif difficulty == "normal":
        move = legal_moves[len(legal_moves) // 2]  # Picks a middle move
    else:  # "hard"
        move = legal_moves[-1]  # Picks the last move (not actually "hard" yet)

    return move.uci()

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "Chess AI Backend Running"}), 200

@app.route("/start_game", methods=["POST"])
def start_game():
    data = request.json
    session_id = data.get("session_id")
    selected_llm = data.get("llm", "gemini")
    difficulty = data.get("difficulty", "normal")

    if not session_id:
        return jsonify({"error": "Session ID is required"}), 400

    with lock:
        games[session_id] = chess.Board()
        move_history[session_id] = []

    return jsonify({
        "message": "Game started",
        "fen": games[session_id].fen(),
        "llm": selected_llm,
        "difficulty": difficulty
    })

@app.route("/make_move", methods=["POST"])
def make_move():
    data = request.json
    session_id = data.get("session_id")
    move_uci = data.get("move")

    if not session_id or session_id not in games:
        return jsonify({"error": "Invalid session ID"}), 400

    board = games[session_id]

    try:
        move = chess.Move.from_uci(move_uci)
    except ValueError:
        return jsonify({"error": "Invalid move format"}), 400

    if move not in board.legal_moves:
        return jsonify({"error": "Illegal move"}), 400

    with lock:
        board.push(move)
        move_history[session_id].append(move_uci)

    if board.is_game_over():
        return jsonify({
            "message": "Game Over",
            "winner": board.result(),
            "history": move_history[session_id]
        })

    selected_llm = data.get("llm", "gemini")
    difficulty = data.get("difficulty", "normal")
    ai_move = get_ai_move(board.fen(), selected_llm, difficulty)

    if ai_move:
        with lock:
            board.push(chess.Move.from_uci(ai_move))
            move_history[session_id].append(ai_move)

    return jsonify({
        "player_move": move_uci,
        "ai_move": ai_move,
        "fen": board.fen(),
        "history": move_history[session_id]
    })

@app.route("/reset_game", methods=["POST"])
def reset_game():
    data = request.json
    session_id = data.get("session_id")

    if not session_id:
        return jsonify({"error": "Session ID is required"}), 400

    with lock:
        games[session_id] = chess.Board()
        move_history[session_id] = []

    return jsonify({
        "message": "Game reset",
        "fen": games[session_id].fen(),
        "history": move_history[session_id]
    })

@app.route("/get_history", methods=["GET"])
def get_history():
    session_id = request.args.get("session_id")

    if not session_id or session_id not in move_history:
        return jsonify({"error": "Invalid session ID"}), 400

    return jsonify({"history": move_history[session_id]})

from flask import Flask
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("0.0.0.0", 5000, app, use_reloader=False, use_debugger=True)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.42:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Apr/2025 16:59:06] "GET / HTTP/1.1" 200 -
192.168.29.42 - - [01/Apr/2025 16:59:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Apr/2025 16:59:27] "GET / HTTP/1.1" 200 -
